In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from sklearn.model_selection import train_test_split
import os
import joblib
import numpy as np
import pandas as pd

In [2]:
ws = Workspace.from_config()
# choose a name for experiment
experiment_name = 'labor-prediction-experiment'
experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath

datastore_name='workspaceblobstore'
datastore=Datastore.get(ws,datastore_name)

datastore_path = [(datastore, 'UI/01-05-2021_070302_UTC/LaborPredictionData6.csv')]
ds = Dataset.Tabular.from_delimited_files(path=datastore_path)

ds = ds.take(3000).to_pandas_dataframe()

In [4]:
from train import clean_data
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=.8)

#y_train=y_train.reshape(1728, 1)
#train = np.concatenate((x_train,y_train), axis=1)
train=pd.concat([x_train, y_train], axis=1)

In [9]:
# TODO: Put your automl settings here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='regression',
    primary_metric='r2_score',
    training_data=train,
    label_column_name='TotalInstallationTime',
    n_cross_validations=10)

In [10]:
#TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_6000a046-d34b-4eac-887f-41b9609706dd

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

***************************************************

In [11]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [37]:
best_mlrun, fitted_mlmodel = remote_run.get_output()
print(best_mlrun)
print(fitted_mlmodel)
joblib.dump(fitted_mlmodel, "automlmodel.pkl")

Run(Experiment: labor-prediction-experiment,
Id: AutoML_6000a046-d34b-4eac-887f-41b9609706dd_40,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             lo

['automlmodel.pkl']

In [38]:
#register the model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice
from azureml.core.model import Model

description = 'Best AutoML Model'
#model = best_mlrun.register_model(description=description, workspace = , model_name='automl_pred', model_path='./models')
model = Model.register(model_path="automlmodel.pkl", model_name = "automl_pred", description=description, workspace=ws)
print(model.name, model.id, model.version, sep='\t')



Registering model automl_pred
automl_pred	automl_pred:7	7


In [39]:
#define inference config
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

#create the environment
myenv = Environment(name="myenv")
conda_dep = CondaDependencies()

#Define the packages needed by the model and scripts
conda_dep.add_conda_package("tensorflow")
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("scikit-learn")
#You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("keras")
conda_dep.add_pip_package("gensim")

myenv.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

In [41]:
#deploy as webservice
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration()

model = Model(ws, name='automl_pred')
service = Model.deploy(ws, 'laborpredservice', [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)
print("scoring URI: " + service.scoring_uri)


Generating Docker build context.
Package creation Succeeded
Logging into Docker registry f9e6381e829d4d91841e9a3e05a328cc.azurecr.io
Logging into Docker registry f9e6381e829d4d91841e9a3e05a328cc.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM f9e6381e829d4d91841e9a3e05a328cc.azurecr.io/azureml/azureml_3fdc9f8c784933dc854a351e73ff60fc
 ---> de09ebaa37e4
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 712f6f2cf5fb
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImNkYmUwYjQzLTkyYTAtNDcxNS04MzhhLWYyNjQ4Y2M3YWQyMSIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTEzMzU0NiIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTEzMzU0NiIsIndvcmtzcGFjZUlkIjoiZjllNjM4MWUtODI5ZC00ZDkxLTg0MWUtOWEzZTA1YTMyOGNjIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 759734eabe0b
 ---> 66656b85256d
Step 4/5 : RUN mv '/var/azureml-app/tmpfbk1c5ud.py' /var/azureml-app/main.py


ERROR:azureml._model_management._util:Error: Container has crashed. Did your init method fail?



WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}

In [ ]:
#send request

import requests
import json

scoring_uri = service.scoring_uri
headers = {'Content-Type':'application/json'}
test_data = {"data": [67, 'ALUMINUM', 'DOOR', 24, 0, 'SNAP BEADS', 9, 118]}
response = requests.post(scoring_uri, data=test_data, headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.json())